In [ ]:
from functools import wraps
import json
import os
import requests
from urllib.parse import urljoin

def openstates_api(api_action):
    API_KEY = os.environ.get("OPENSTATES_API_KEY")
    
    @wraps(api_action)
    def api_wrapper(*args, **kwargs):
        if 'api_key' in kwargs and kwargs['api_key']:
            return api_action(*args, **kwargs)
        
        return api_action(*args, **(kwargs | {'api_key': API_KEY}))
                          
    return api_wrapper

with open('../artifacts/openstates_jurisdictions.json', 'r') as f:
    jurisdictions = json.load(f)

    
@openstates_api
def search_bill(state: str, year: str, bill_id: str, api_key: str):
    API_ROOT = 'https://v3.openstates.org/'
    ENDPOINT = 'bills'

    jurisdiction = jurisdictions[state]
    
    params = {
        'jurisdiction': jurisdiction['id'],
        'session': year,
        'identifier': bill_id,
    }
    
    headers = {
        'x-api-key': api_key
    }
    
    resp = requests.get(urljoin(API_ROOT, ENDPOINT), params=params, headers=headers)
    return resp.text

search_bill('FL', '2023', 'HB 1069')

In [ ]:
from itertools import chain
import json

with open('../datasets/geography.json', 'r') as f:
    geography = json.load(f)
    
state_abbreviations = geography['state_abbreviations']

with open('../datasets/openstates_country_jurisdictions.json', 'r') as f:
    states = json.load(f)
    
with open('../datasets/openstates_state_jurisdictions.json', 'r') as f:
    countries = json.load(f)

all_jurisdiction_results = chain(states['results'], countries['results'])
    
openstates_jurisdictions = {
    jurisdiction['name']: jurisdiction
    for jurisdiction
    in all_jurisdiction_results
}

openstates_jurisdictions.update({
    state_abbreviations[k.replace(' of ', ' Of ')]: v
    for k, v
    in openstates_jurisdictions.items()
})

with open('../artifacts/openstates_jurisdictions.json', 'w') as f:
    json.dump(openstates_jurisdictions, f, indent=2)


In [ ]:
from functools import wraps
import json
import os
import requests
from urllib.parse import urljoin

def openstates_api(api_action):
    API_KEY = os.environ.get("OPENSTATES_API_KEY")
    
    @wraps(api_action)
    def api_wrapper(*args, **kwargs):
        if 'api_key' in kwargs and kwargs['api_key']:
            return api_action(*args, **kwargs)
        
        return api_action(*args, **(kwargs | {'api_key': API_KEY}))
                          
    return api_wrapper

@openstates_api
def get_jurisdictions(scope: str, api_key: str):
    API_ROOT = 'https://v3.openstates.org/'
    ENDPOINT = 'jurisdictions'
    
    params = {
        'classification': scope,
    }
    
    headers = {
        'x-api-key': api_key
    }
    
    resp = requests.get(urljoin(API_ROOT, ENDPOINT), params=params, headers=headers)
    return resp.text

with open('../datasets/openstates_state_jurisdictions.json', 'w') as f:
    f.write(get_jurisdictions('state'))
with open('../datasets/openstates_country_jurisdictions.json', 'w') as f:
    f.write(get_jurisdictions('country'))